# VGG16

## Transfer learning on Our DataSet

In [6]:
import numpy as np
import os
import time
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
# Loading the training data
data_path = r'E:/surbhi/phd/Sem2/ML/code/dataset'
data_dir_list = os.listdir(data_path)

img_data_list=[]
j=0
for dataset in data_dir_list:
    print(dataset)
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    i=1

    for img in img_list:
        if i<=200:
            img_path = data_path + '/'+ dataset + '/'+ img
            img = image.load_img(img_path, target_size=(224, 224))
            x = image.img_to_array(img)
            #print(x,x.shape)
            x = np.expand_dims(x, axis=0)
            #print('xp ',x,x.shape)
            x = preprocess_input(x)
            x = x/255
            img_data_list.append(x)
            i+=1
        else:
            break
        j+=1
    print(j)

img_data = np.array(img_data_list)
#img_data = img_data.astype('float32')
print (img_data.shape)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)
img_data=img_data[0]
print (img_data.shape)

down_left
Loaded the images of dataset-down_left

200
down_right
Loaded the images of dataset-down_right

400
up_left
Loaded the images of dataset-up_left

600
up_right
Loaded the images of dataset-up_right

800
(800, 1, 224, 224, 3)
(1, 800, 224, 224, 3)
(800, 224, 224, 3)


In [8]:
# Define the number of classes
num_classes = 4
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,),dtype='int64')

labels[0:200]=0    #down left
labels[200:400]=1  #down_right
labels[400:600]=2  #up_left
labels[600:]=3     #up_right

names = ['DL','DR','UL','UR']

# convert class labels to on-hot encoding
Y = np_utils.to_categorical(labels, num_classes)

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(640, 224, 224, 3) (160, 224, 224, 3) (640, 4) (160, 4)


In [9]:
from keras.applications.vgg16 import VGG16
image_input = Input(shape=(224, 224, 3))
tf_model = VGG16(input_tensor=image_input,include_top=True)
print(tf_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

#### Changed model (last dense layers)

In [10]:
last_layer = tf_model.get_layer('block5_pool').output
x= Flatten(name='flatten')(last_layer)
dense1 = Dense(128, activation='relu', name='dense1')(x)
dense2 = Dense(256, activation='relu', name='dense2')(dense1)
out = Dense(4, activation='softmax', name='output')(dense2)

custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
for layer in custom_vgg_model.layers[:-3]:
    layer.trainable = False

custom_vgg_model.layers[3].trainable

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [56]:
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=20, verbose=1, validation_data=(X_test, y_test))
fname = 'tfvggweight3.hdf5'
model_json = custom_vgg_model.to_json()
with open("tfvggmodel3.json", "w") as json_file:
    json_file.write(model_json)
custom_vgg_model.save_weights(fname,overwrite=True)

Train on 640 samples, validate on 160 samples
Epoch 1/20
640/640 [==============================] - 282s 440ms/step - loss: 1.0241 - acc: 0.6094 - val_loss: 0.4936 - val_acc: 0.7937
Epoch 2/20
640/640 [==============================] - 277s 432ms/step - loss: 0.3555 - acc: 0.8516 - val_loss: 0.4100 - val_acc: 0.8562
Epoch 3/20
640/640 [==============================] - 274s 428ms/step - loss: 0.1829 - acc: 0.9344 - val_loss: 0.2445 - val_acc: 0.9062
Epoch 4/20
640/640 [==============================] - 275s 429ms/step - loss: 0.1163 - acc: 0.9547 - val_loss: 0.2135 - val_acc: 0.9000
Epoch 5/20
640/640 [==============================] - 274s 429ms/step - loss: 0.0686 - acc: 0.9781 - val_loss: 0.2113 - val_acc: 0.9187
Epoch 6/20
640/640 [==============================] - 276s 431ms/step - loss: 0.0616 - acc: 0.9812 - val_loss: 0.3579 - val_acc: 0.8625
Epoch 7/20
640/640 [==============================] - 275s 430ms/step - loss: 0.0536 - acc: 0.9812 - val_loss: 0.1789 - val_acc: 0.9250
Ep

In [14]:
from keras.models import model_from_json
import cv2
import os
def predict():
    json_file = open('tfvggmodel3.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)

    model.load_weights('tfvggweight3.hdf5')
    #print(model.summary())
    path=r'E:/surbhi/phd/Sem2/ML/code/test'

    for file in os.listdir(path):
        print('Actual O/P:',file)
        img=cv2.imread(os.path.join(path,file))
        img=cv2.resize(img,(224,224))
        #print(img.shape)
        img = img.reshape(1, 224,224,3)
        #print(img.shape)
        img = img.astype('float32')
        img =img / 255
        y_proba = model.predict(img)
        y_classes = y_proba.argmax(axis=-1)
        print(y_classes)

predict()

Actual O/P: dl1 (1).jpg
[0]
Actual O/P: dl1 (2).jpg
[0]
Actual O/P: dl1 (3).jpg
[0]
Actual O/P: dl1 (4).jpg
[0]
Actual O/P: dl1 (5).jpg
[0]
Actual O/P: dr (1).jpg
[1]
Actual O/P: dr (2).jpg
[1]
Actual O/P: dr (3).jpg
[1]
Actual O/P: dr (4).jpg
[1]
Actual O/P: i l.jfif
[1]
Actual O/P: ul (1).jpg
[2]
Actual O/P: ul (2).jpg
[2]
Actual O/P: ul (3).jpg
[1]
Actual O/P: ul (4).jpg
[2]
Actual O/P: ur (1).jpg
[3]
Actual O/P: ur (2).jpg
[3]
Actual O/P: ur (3).jpg
[3]
Actual O/P: ur (4).jpg
[3]
Actual O/P: ur5.jfif
[2]
